In [368]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from src.dh import DHFrame
import src.tools as tools
import sympy as sp

np.set_printoptions(suppress=True)

In [369]:
r1,r2,r3,r4,a1,a2 = 0.4, 0.455, 0.420, 0.08, 0.025, 0.035
r3_o = np.sqrt(r3**2 + a2**2) # the distance between joint 3 and the wrist
Slist = np.array([[0,0,-1,0,0,0],
                  [0,1,0,-r1,0,a1],
                  [0,1,0,-r1,0,(r2+a1)],
                  [-1,0,0,0,-(r1+a2),0],
                  [0,1,0,-(r1+a2),0,(r2+r3+a1)],
                  [-1,0,0,0,-(r1+a2),0]]).T

M = np.array([[0,0,1,r2+r3+r4+a1],
              [0,1,0,0],
              [-1,0,0,r1+a2],
              [0,0,0,1]])

In [370]:

# thetalistTest = np.array([33,-63,-34,15,33,33])*np.pi/180
thetalistTest = np.array([0,0,0,0,0,0])*np.pi/180
thetalistTest = np.array([0,-90,90,0,0,0])*np.pi/180
thetalistTest = np.array([45,-90,10,0,30,0])*np.pi/180
# thetalistTest = np.array([-90,-90,90,30,30,30])*np.pi/180 #Write the thetas to make a 
                                                        #transformationmatrix T_SB to
                                                        #test the analytical solver

T_SB = mr.FKinSpace(M,Slist,thetalistTest)


#Translate -80mm along z-axis of endframe

T_BW = np.array([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,-0.08],
                 [0,0,0,1]])

T_SW = np.dot(T_SB,T_BW)
print(T_SW.round(2))


pw = np.array([T_SW[0][3],T_SW[1][3],T_SW[2][3]])
theta11 = -np.arctan2(pw[1],pw[0])
theta12 = -np.arctan2(-pw[1],-pw[0])




[[ 0.54  0.71  0.45  0.04]
 [-0.54  0.71 -0.45 -0.04]
 [-0.64  0.    0.77  1.27]
 [ 0.    0.    0.    1.  ]]


In [371]:
#Forward

Px, Py, Pz =pw[0] - np.cos(theta11) * a1, \
            pw[1] + np.sin(theta11) * a1, \
            pw[2] - r1


c3 = (Px**2 + Py**2 + Pz**2 - r2**2 - r3_o**2) / (2 * r2 * r3_o)

s3p = np.sqrt(1-c3**2)
s3n = -s3p

theta31 = np.arctan2(s3p,c3) + np.arctan2(a2,r3)
theta32 = np.arctan2(s3n,c3) + np.arctan2(a2,r3)

c2pp = (r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3p * Pz
c2np = -(r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3p * Pz
c2pn = (r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3n * Pz
c2nn = -(r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3n * Pz

s2np = (r2 + r3_o*c3) * Pz - r3_o * s3p * np.sqrt(Px**2 + Py**2)
s2pp = (r2 + r3_o*c3) * Pz + r3_o * s3p * np.sqrt(Px**2 + Py**2)
s2nn = (r2 + r3_o*c3) * Pz - r3_o * s3n * np.sqrt(Px**2 + Py**2)
s2pn = (r2 + r3_o*c3) * Pz + r3_o * s3n * np.sqrt(Px**2 + Py**2)

theta21 = np.arctan2(s2np,c2pp) 
# theta22 = np.arctan2(s2pp,c2np)
theta23 = np.arctan2(s2nn,c2pn) 
# theta24 = np.arctan2(s2pn,c2nn) 

In [372]:
#Backward
Px, Py, Pz =pw[0] - np.cos(theta12) * a1, \
            pw[1] + np.sin(theta12) * a1, \
            pw[2] - r1

c3 = (Px**2 + Py**2 + Pz**2 - r2**2 - r3_o**2) / (2 * r2 * r3_o)

s3p = np.sqrt(1-c3**2)
s3n = -s3p

theta33 = np.arctan2(s3p,c3) + np.arctan2(a2,r3)
theta34 = np.arctan2(s3n,c3) + np.arctan2(a2,r3)

c2pp = (r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3p * Pz
c2np = -(r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3p * Pz
c2pn = (r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3n * Pz
c2nn = -(r2 + r3_o * c3) * np.sqrt(Px**2 + Py**2) + r3_o * s3n * Pz

s2np = (r2 + r3_o*c3) * Pz - r3_o * s3p * np.sqrt(Px**2 + Py**2)
s2pp = (r2 + r3_o*c3) * Pz + r3_o * s3p * np.sqrt(Px**2 + Py**2)
s2nn = (r2 + r3_o*c3) * Pz - r3_o * s3n * np.sqrt(Px**2 + Py**2)
s2pn = (r2 + r3_o*c3) * Pz + r3_o * s3n * np.sqrt(Px**2 + Py**2)

# theta21 = np.arctan2(s2np,c2pp) 
theta22 = np.arctan2(s2pp,c2np)
# theta23 = np.arctan2(s2nn,c2pn) 
theta24 = np.arctan2(s2pn,c2nn) 




config = np.array([[theta11, -theta23, theta31], # forward over elbow
                  [theta11,-theta21, theta32], # forward under elbow
                  [theta12, -theta24, theta33], # backward under elbow
                  [theta12, -theta22, theta34]]) # backward over elbow
config*180/np.pi

C:\Users\Simen\AppData\Local\Temp/ipykernel_15760/1347141614.py:8: RuntimeWarning: invalid value encountered in sqrt
  s3p = np.sqrt(1-c3**2)


array([[  45.        ,  -90.        ,   10.        ],
       [  45.        ,  -84.96418983,   -0.47271662],
       [-135.        ,           nan,           nan],
       [-135.        ,           nan,           nan]])

In [373]:
for i in config:
    test = np.array([i[0],i[1],i[2],0,0,0])
    print(test*180/np.pi)
    print(np.allclose(tools.pointFromT(T_SW), tools.pointFromT(np.dot(mr.FKinSpace(M,Slist,test), T_BW))))

[ 45. -90.  10.   0.   0.   0.]
True
[ 45.         -84.96418983  -0.47271662   0.           0.
   0.        ]
True
[-135.   nan   nan    0.    0.    0.]
False
[-135.   nan   nan    0.    0.    0.]
False


Thinking of the wrist as a -ZY-Z Euler config

In [379]:
# for i in config[:1,:]:
thetalist = np.zeros(shape=(4,6))
thetalist2 = np.zeros(shape=(4,6))
for i, theta in enumerate(config):
    t1,t2,t3 = theta[0],theta[1],theta[2]
    R_S3 = tools.rotz(t1) @ tools.roty(t2) @ tools.roty(t3) @ M[:3,:3]

    R_SB = T_SB[:3,:3]
    R_3B = R_S3.T @ R_SB

    theta61 = np.arctan2(R_3B[2,1],-R_3B[2,0])
    theta62 = np.arctan2(-R_3B[2,1],R_3B[2,0])

    theta51 = np.arctan2(np.sqrt(1-R_3B[2,2]**2), R_3B[2,2])
    theta52 = np.arctan2(-np.sqrt(1-R_3B[2,2]**2), R_3B[2,2])
    
    theta41 = np.arctan2(R_3B[1,2],R_3B[0,2])
    theta42 = np.arctan2(-R_3B[1,2],-R_3B[0,2])

    thetalist[i] = np.concatenate((theta,np.array([theta41,theta51,theta61])))
    thetalist2[i] = np.concatenate((theta,np.array([theta42,theta52,theta62])))
    # print((thetalist*180/np.pi).round(2))
    # print((thetalist2*180/np.pi).round(2))
    # print(mr.FKinSpace(M,Slist,thetalist))
    # print(mr.FKinSpace(M,Slist,thetalist2))
    # print(np.isclose(T_SB,mr.FKinSpace(M,Slist,thetalist)))
    # print(np.isclose(T_SB,mr.FKinSpace(M,Slist,thetalist2)))

array([[ 0.78539816, -1.57079633,  0.17453293, -1.77486157,  0.71604683,
         0.26773027],
       [ 0.78539816, -1.48290486, -0.00825046, -1.6653259 ,  0.70190071,
         0.12352983],
       [-2.35619449,         nan,         nan,         nan,         nan,
                nan],
       [-2.35619449,         nan,         nan,         nan,         nan,
                nan]])

In [375]:
mr.FKinSpace(M,Slist,thetalist[0])

array([[ 0.54167522, -0.70710678,  0.45451948,  0.08123736],
       [ 0.54167522,  0.70710678,  0.45451948, -0.00851424],
       [-0.64278761,  0.        ,  0.76604444,  1.3359805 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [376]:
mr.FKinSpace(M,Slist,thetalist2[0])

array([[ 0.54167522, -0.70710678,  0.45451948,  0.08123736],
       [ 0.54167522,  0.70710678,  0.45451948, -0.00851424],
       [-0.64278761, -0.        ,  0.76604444,  1.3359805 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [377]:
T_SB

array([[ 0.54167522,  0.70710678,  0.45451948,  0.08123736],
       [-0.54167522,  0.70710678, -0.45451948, -0.08123736],
       [-0.64278761,  0.        ,  0.76604444,  1.3359805 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])